In [59]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
df=pd.read_csv('/content/drive/MyDrive/shuttle-c0-vs-c4.csv')
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,49,0,79,0,46,-5,30,32,2,negative
1,41,1,86,3,42,6,45,45,0,negative
2,50,-4,83,0,50,0,33,34,2,negative
3,48,1,81,-1,46,0,33,34,2,negative
4,45,0,86,0,44,-10,41,42,2,negative
...,...,...,...,...,...,...,...,...,...,...
1824,75,0,80,4,-42,0,5,124,120,positive
1825,85,0,88,0,2,-17,3,86,82,positive
1826,81,0,86,0,-40,2,4,127,122,positive
1827,84,0,88,0,-2,-12,3,90,88,positive


In [62]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0       negative
1       negative
2       negative
3       negative
4       negative
          ...   
1824    positive
1825    positive
1826    positive
1827    positive
1828    positive
Name: Class, Length: 1829, dtype: object

In [63]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,49,0,79,0,46,-5,30,32,2,0
1,41,1,86,3,42,6,45,45,0,0
2,50,-4,83,0,50,0,33,34,2,0
3,48,1,81,-1,46,0,33,34,2,0
4,45,0,86,0,44,-10,41,42,2,0


In [64]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(123, 9)

In [65]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(1706, 9)

In [66]:
df_minor = pd.DataFrame(minority, columns =['A1','A2','A3','A4','A5','A6','A7','A8','A9'])
df_minor

,A1,A2,A3,A4,A5,A6,A7,A8,A9
0,81.000000,-5.000000,84.000000,0.000000,-22.000000,-6.000000,4.0,108.000000,104.000000
1,103.000000,-1.000000,104.000000,-4.000000,72.000000,10.000000,1.0,31.000000,30.000000
2,86.000000,5.000000,89.000000,0.000000,2.000000,-5.000000,3.0,86.000000,84.000000
3,106.000000,0.000000,108.000000,-3.000000,70.000000,-4.000000,1.0,38.000000,36.000000
4,79.000000,0.000000,83.000000,0.000000,-46.000000,-29.000000,4.0,130.000000,126.000000
...,...,...,...,...,...,...,...,...,...
1701,123.000000,7.470182,108.000000,-33.376606,18.135811,16.376900,5.0,130.000000,126.000000
1702,123.000000,30.330415,108.000000,10.000000,23.689137,2.691067,5.0,130.000000,99.992625
1703,113.175522,2.997260,108.000000,10.000000,80.275328,8.438019,5.0,48.482193,44.788045
1704,121.481242,-5.000000,103.833605,10.000000,-38.237341,2.473725,5.0,130.000000,126.000000


In [67]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,81.000000,-5.000000,84.000000,0.000000,-22.000000,-6.000000,4.0,108.000000,104.000000,1
1,103.000000,-1.000000,104.000000,-4.000000,72.000000,10.000000,1.0,31.000000,30.000000,1
2,86.000000,5.000000,89.000000,0.000000,2.000000,-5.000000,3.0,86.000000,84.000000,1
3,106.000000,0.000000,108.000000,-3.000000,70.000000,-4.000000,1.0,38.000000,36.000000,1
4,79.000000,0.000000,83.000000,0.000000,-46.000000,-29.000000,4.0,130.000000,126.000000,1
...,...,...,...,...,...,...,...,...,...,...
1701,123.000000,7.470182,108.000000,-33.376606,18.135811,16.376900,5.0,130.000000,126.000000,1
1702,123.000000,30.330415,108.000000,10.000000,23.689137,2.691067,5.0,130.000000,99.992625,1
1703,113.175522,2.997260,108.000000,10.000000,80.275328,8.438019,5.0,48.482193,44.788045,1
1704,121.481242,-5.000000,103.833605,10.000000,-38.237341,2.473725,5.0,130.000000,126.000000,1


In [68]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','Class'])
df_major

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,49,0,79,0,46,-5,30,32,2,0
1,41,1,86,3,42,6,45,45,0,0
2,50,-4,83,0,50,0,33,34,2,0
3,48,1,81,-1,46,0,33,34,2,0
4,45,0,86,0,44,-10,41,42,2,0
...,...,...,...,...,...,...,...,...,...,...
1701,45,0,84,0,44,-6,40,41,2,0
1702,54,0,108,5,54,0,54,54,0,0
1703,41,0,84,0,42,17,44,43,0,0
1704,37,0,95,1,10,5,57,84,28,0


In [69]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,81.0,-5.0,84.0,0.0,-22.0,-6.0,4.0,108.0,104.0,1
1,103.0,-1.0,104.0,-4.0,72.0,10.0,1.0,31.0,30.0,1
2,86.0,5.0,89.0,0.0,2.0,-5.0,3.0,86.0,84.0,1
3,106.0,0.0,108.0,-3.0,70.0,-4.0,1.0,38.0,36.0,1
4,79.0,0.0,83.0,0.0,-46.0,-29.0,4.0,130.0,126.0,1
...,...,...,...,...,...,...,...,...,...,...
3407,45.0,0.0,84.0,0.0,44.0,-6.0,40.0,41.0,2.0,0
3408,54.0,0.0,108.0,5.0,54.0,0.0,54.0,54.0,0.0,0
3409,41.0,0.0,84.0,0.0,42.0,17.0,44.0,43.0,0.0,0
3410,37.0,0.0,95.0,1.0,10.0,5.0,57.0,84.0,28.0,0


In [70]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [71]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [72]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [1.0, 0.9970674486803519, 0.9970631487795462, 0.9970631487795462, 0.9970631487795462]
AUC_SVM: [1.0, 0.9985380116959065, 0.998533724340176, 0.998533724340176, 0.998533724340176]
f1_SVM: [1.0, 0.9985358711566619, 0.9985358711566619, 0.9985358711566619, 0.9985315712187959]
precision_SVM: [1.0, 0.9970760233918129, 0.9970760233918129, 0.9970760233918129, 1.0]
recall_SVM: [1.0, 1.0, 1.0, 1.0, 0.9970674486803519]
mean of Gmean_s: 0.9976513790037981
mean of AUC_s: 0.9988278369432869
mean of f1_s: 0.9988278369377563
mean of precision_s: 0.9982456140350877
mean of recall_s: 0.9994134897360704
standard deviation of Gmean_s: 0.0013129203704538635
standard deviation of AUC_s: 0.0006552616987639742
standard deviation of f1_s: 0.0006552617173141407
standard deviation of precision_s: 0.001601527945921523
standard deviation of recall_s: 0.0013114768196479693


-------KNN--------
Gmean_KNN: [1.0, 0.9970674486803519, 0.9970631487795462, 1.0, 1.0]
AUC_KNN: [1.0, 0.9985380116

In [73]:
list=['shuttle0vs4',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836


In [74]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')